In [2]:
import webbrowser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_bootstrap_components as dbc
import dash_ag_grid as dag

app = dash.Dash(__name__)

df = pd.read_csv("./data/atp_matches_2024.csv")
winners = df['winner_name'].unique()

app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    dash.dcc.Dropdown(
                        id="player",
                        options=[{"label": name, "value": name} for name in winners],
                        value=winners[0],
                    ),
                    md=4,
                ),
                dbc.Col(
                    [
                        dash.dcc.Graph(
                            id="chart",
                            figure={},
                            style={"width": "50%", "margin": "auto"},
                        ),
                        dash.dcc.Graph(
                            id="chart2",
                            figure={},
                            style={"width": "50%", "margin": "auto"},
                        ),
                    ],
                    md=4,
                    style={"textAlign": "center", "display": "flex"},
                ),
                dbc.Col(
                    [
                        dag.AgGrid(
                            id="table",
                            columnDefs=[{"field": i} for i in df.columns],
                            rowData=[],  # Start with empty data
                        )
                    ],
                    md=4,
                ),
            ]
        )
    ],
    style={"font-family": "Rubik"},
)

@app.callback(
    dash.Output('chart', 'figure'),
    dash.Output('table', 'rowData'),
    dash.Output('table', 'defaultColDef'),
    dash.Output('chart2', 'figure'),
    dash.Input('player', 'value')
)
def update_chart(player):
    if player is None:
        return dash.no_update, dash.no_update, dash.no_update

    player_stats_win = df[df['winner_name'] == player]
    player_stats_lose = df[df['loser_name'] == player]

    win_count = len(player_stats_win)
    lose_count = len(player_stats_lose)

    # Wins by tournament
    tourney_counts_win = player_stats_win["tourney_name"].value_counts()
    # Losses by tournament
    tourney_counts_lose = player_stats_lose["tourney_name"].value_counts()

    # Align indexes for plotting
    all_tourneys = tourney_counts_win.index.union(tourney_counts_lose.index)
    win_vals = tourney_counts_win.reindex(all_tourneys, fill_value=0)
    lose_vals = tourney_counts_lose.reindex(all_tourneys, fill_value=0)

    stacked_bar_chart = {
        "data": [
            {
                "type": "bar",
                "name": "Wins",
                "x": all_tourneys.tolist(),
                "y": win_vals.tolist(),
                "marker": {"color": "#00AEFF"},
            },
            {
                "type": "bar",
                "name": "Losses",
                "x": all_tourneys.tolist(),
                "y": lose_vals.tolist(),
                "marker": {"color": "#FF7E70"},
            },
        ],
        "layout": {
            "title": f"{player} - Number of Wins and Losses by Tournament (2024)",
            "xaxis": {"title": "Tournament", "tickangle": -45},
            "yaxis": {
                "title": "Number of Matches",
                "tickformat": "d",  # Show only full numbers
            },
            "margin": {"b": 120},
            "barmode": "stack",
            "font": {"family": "Rubik"},
        },
    }

    pie_chart = {
        "data": [
            {
                "type": "pie",
                "labels": ["Wins", "Losses"],
                "values": [win_count, lose_count],
                "hole": 0.7,  # optional: makes it a donut chart
                "marker": {"colors": ["#00AEFF", "#FF7E70"]},
            }
        ],
        "layout": {
            "title": f"{player} - Wins vs Losses (2024)",
            "font": {"family": "Rubik"},
        },
    }

    cell_styling = {
        "styleConditions": [
            {
                "condition": "params.colDef.field == 'Tourney_level' && params.value == 'A'",
                "style": {"backgroundColor": "lightblue", "color": "black"},
            }
        ]
    }
    return stacked_bar_chart, player_stats_win.to_dict('records'), cell_styling, pie_chart

if __name__ == "__main__":
    url = "http://127.0.0.1:8050/"
    webbrowser.open(url)  # opens default browser
    app.run(debug=True)